# VacationPy


In [63]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

#To display all outputs in a call
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
DF = pd.read_csv('../Output/cleaned_cities_weather.csv', encoding='utf-8')


In [4]:
DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 584 entries, 0 to 583
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City        584 non-null    object 
 1   Country     579 non-null    object 
 2   Lat         584 non-null    float64
 3   Lng         584 non-null    float64
 4   Max Temp    584 non-null    float64
 5   Humidity    584 non-null    float64
 6   Cloudiness  584 non-null    float64
 7   Wind Speed  584 non-null    float64
dtypes: float64(6), object(2)
memory usage: 36.6+ KB


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [51]:
gmaps.configure(api_key=g_key)
layout = {
    'width': '1000px',
    'height': '1000px',
    'border': '1px solid black',
    'padding': '0px',
    'margin': '0'
}
Map = gmaps.figure(center=(0, 0), zoom_level=2, layout=layout)
heatmap_layer = gmaps.WeightedHeatmap(locations = DF[['Lat', 'Lng']], weights=DF['Humidity'], max_intensity=DF['Humidity'].max())
Map.add_layer(heatmap_layer)

In [52]:
from ipywidgets.embed import embed_minimal_html
embed_minimal_html('heatmap.html', views=[Map], title='heatmap')

In [53]:
Map

Figure(layout=FigureLayout(border='1px solid black', height='1000px', margin='0', padding='0px', width='1000px…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

**Ideal Weather Conditions**

* A max temperature lower than 80 Fahrenheit but higher than 70 Fahrenheit.

* Wind speed less than 10 mph.

* Zero cloudiness.

Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

In [93]:
DF_IDEAL = DF.drop(DF[~((DF['Max Temp']<80.0) & (DF['Max Temp']>70.0) & (DF['Wind Speed']<10.0) & (DF['Cloudiness']==0))].index)

DF_IDEAL.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14 entries, 25 to 536
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City        14 non-null     object 
 1   Country     14 non-null     object 
 2   Lat         14 non-null     float64
 3   Lng         14 non-null     float64
 4   Max Temp    14 non-null     float64
 5   Humidity    14 non-null     float64
 6   Cloudiness  14 non-null     float64
 7   Wind Speed  14 non-null     float64
dtypes: float64(6), object(2)
memory usage: 1008.0+ bytes


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [127]:
hotel_df = DF_IDEAL.iloc[:,:4].copy()
hotel_df['Hotel Name'] = ""

base_url = 'https://maps.googleapis.com/maps/api/place/textsearch/json'

for index, row in hotel_df.iterrows():
    
    
    params = {
            "location": f"{row['Lat']},{row['Lng']}",
            "query": 'hotel',
            "radius": 5000,
            "key": g_key
            }
    
    try:
        result = requests.get(base_url, params).json()
        hotel_df.loc[index, "Hotel Name"] = result['results'][0]['name']
        
    except:
        print(f"Couldn't retrive hotel for {row['City']} at index {index}..Skipping")

In [128]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
25,Maţāy,EG,28.42,30.78,ABO GHANIMA HOME
37,São Domingos do Maranhão,BR,-5.58,-44.39,Pousada Pontes
51,São Filipe,CV,14.90,-24.50,Hotel Xaguate
60,Kurumkan,RU,54.30,110.30,Gostevoy Dom Vista
70,Tiznit Province,MA,29.58,-9.50,Hotel kerdous
113,Cherskiy,RU,68.75,161.30,Super 8 by Wyndham Austin/Airport North
144,Elesbão Veloso,BR,-6.20,-42.14,Pousada Rodeio
149,Amga,RU,60.90,131.96,Algys
201,Diamantino,BR,-14.41,-56.45,KAYABI PALACE HOTEL E RESTAURANTE
241,Verkh-Usugli,RU,52.70,115.17,"Gostinitsa ""Aleksandriya"""


In [129]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [143]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, hover_text='', label='', info_box_content=hotel_info, display_info_box=True)
Map.add_layer(marker_layer)

#Save as html
from ipywidgets.embed import embed_minimal_html
embed_minimal_html('hotelmap.html', views=[Map], title='hotelmap')
# Display figure
Map

Figure(layout=FigureLayout(border='1px solid black', height='1000px', margin='0', padding='0px', width='1000px…